# Agent with Memory Tools

In this notebook, you'll build an agent that has **explicit memory tools** alongside the context provider. While the context provider silently injects relevant context, memory tools let the agent actively search, save, and recall information.

The `create_memory_tools()` function creates callable `FunctionTool` instances:
- `search_memory` — Search across all memory types
- `remember_preference` — Save a user preference
- `recall_preferences` — Recall saved preferences
- `search_knowledge` — Search the knowledge graph for entities
- `remember_fact` — Save a factual statement
- `find_similar_tasks` — Find similar past reasoning traces

***

Load the environment variables and import the required modules.

In [ ]:
import sys
sys.path.insert(0, '../shared')

import asyncio

from pydantic import SecretStr

from agent_framework.azure import AzureAIClient
from azure.identity.aio import AzureCliCredential

from neo4j_agent_memory import MemoryClient, MemorySettings
from neo4j_agent_memory.integrations.microsoft_agent import (
    Neo4jMicrosoftMemory,
    create_memory_tools,
)

from config import get_agent_config, Neo4jConfig

## Set Up Memory with Tools

Create the memory client and unified memory interface, then generate callable memory tools.

In [ ]:
# Get workshop configuration
config = get_agent_config()
neo4j_config = Neo4jConfig()

# Configure and create memory client
settings = MemorySettings(
    neo4j={
        "uri": neo4j_config.uri,
        "username": neo4j_config.username,
        "password": SecretStr(neo4j_config.password),
    },
)

memory_client = MemoryClient(settings)
await memory_client.__aenter__()

# Create unified memory
memory = Neo4jMicrosoftMemory.from_memory_client(
    memory_client=memory_client,
    session_id="workshop-tools-demo",
    include_short_term=True,
    include_long_term=True,
    extract_entities=True,
)

# Create callable memory tools
tools = create_memory_tools(memory)

print(f"Created {len(tools)} memory tools:")
for t in tools:
    print(f"  - {t.name}: {t.description[:60]}...")

> These tools are `FunctionTool` instances that the Microsoft Agent Framework auto-invokes during streaming. The agent decides when to use each tool based on the tool's name and description.

***

## Create the Agent with Memory Tools and Context Provider

Combine the context provider (automatic background context) with memory tools (explicit agent-driven operations).

In [ ]:
async def run_memory_agent():
    async with AzureCliCredential() as credential:
        async with AzureAIClient(
            project_endpoint=config.project_endpoint,
            model_deployment_name=config.model_name,
            async_credential=credential,
        ) as client:
            async with client.create_agent(
                name="workshop-memory-tools-agent",
                instructions=(
                    "You are a helpful assistant with persistent memory. You have access to "
                    "memory tools that let you:\n"
                    "1. Search your memory for relevant past conversations and facts\n"
                    "2. Save user preferences when they express them\n"
                    "3. Recall preferences to personalize your responses\n"
                    "4. Search the knowledge graph for entities\n"
                    "5. Remember important facts for future reference\n\n"
                    "Always use the appropriate memory tools to provide personalized assistance. "
                    "When the user expresses a preference, save it. When making recommendations, "
                    "recall their preferences first."
                ),
                tools=tools,
                context_providers=[memory.context_provider],
            ) as agent:
                session = agent.create_session()

                queries = [
                    "I prefer concise technical explanations over high-level overviews.",
                    "What can you tell me about supply chain risks in tech companies?",
                    "Remember that I'm particularly interested in Apple and Microsoft.",
                    "Based on what you know about my preferences, what should I focus on?",
                ]

                for query in queries:
                    print(f"User: {query}\n")
                    print("Assistant: ", end="", flush=True)

                    async for update in agent.run_stream(query, session=session):
                        if update.text:
                            print(update.text, end="", flush=True)

                    print("\n\n" + "-" * 50 + "\n")

    await asyncio.sleep(0.1)

await run_memory_agent()

> The agent used memory tools to save preferences and recall them later. The context provider also injected relevant conversation history automatically.

***

## Verify Stored Memories

Let's check what the agent stored in memory.

In [ ]:
# Search for stored preferences
results = await memory.search_memory(
    query="user preferences and interests",
    include_messages=True,
    include_entities=True,
    include_preferences=True,
    limit=5,
)

print("=== Stored Memories ===\n")

if results.get("preferences"):
    print("Preferences:")
    for pref in results["preferences"]:
        print(f"  [{pref['category']}] {pref['preference']}")
    print()

if results.get("entities"):
    print("Entities:")
    for entity in results["entities"][:5]:
        print(f"  {entity['name']} ({entity['type']})")
    print()

if results.get("messages"):
    print(f"Messages stored: {len(results['messages'])}")

***

Experiment with the agent:
* Express different preferences and see if the agent remembers them
* Ask the agent to recall what it knows about you
* Try multi-turn conversations to test memory persistence

***

[View the complete code](../financial_data_load/solution_srcs/06_02_memory_tools_agent.py)

[Continue to Lab 8 - Building a Knowledge Graph](../Lab_8_Knowledge_Graph)

In [ ]:
# Cleanup
await memory_client.__aexit__(None, None, None)